In [13]:
from InterOptimus.itworker import InterfaceWorker
from pymatgen.core.structure import Structure
from mp_api.client import MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

# Input film & substrate CONVENTIONAL structure

In [ ]:
with MPRester('fFtrdShVJH4jwWHiId8v4cyGzV2oYnoG') as mpr:
        docs = mpr.materials.summary.search(
        material_ids = ["mp-1153", "mp-362"],
        fields=["material_id", "structure", "nelements"])
        for doc in docs:
            material_id = doc.material_id
            structure = doc.structure
    
            # 使用 SpacegroupAnalyzer 获取常规结构
            analyzer = SpacegroupAnalyzer(structure)
            doc.structure = analyzer.get_conventional_standard_structure()
for i in docs:
    if i.material_id == "mp-1153":
        film_conv = i.structure
    else:
        substrate_conv = i.structure
iw = InterfaceWorker(film_conv, substrate_conv)

# Define InterfaceWorker

In [ ]:
iw = InterfaceWorker(film_conv, substrate_conv)

# lattice matching

In [ ]:
iw.lattice_matching(max_area = 47, max_length_tol = 0.03, max_angle_tol = 0.03,
                    film_max_miller = 1, substrate_max_miller = 1, film_millers = None, substrate_millers = None)
#Note max millers are in primitive cell, should be different from the indices in the polar projection below which are in conventional cell
#If you specify miller indices yourself, just ignore film_max_miller and substrate_max_miller

In [ ]:
#Visualize matching information
iw.ems.plot_unique_matches()

In [ ]:
iw.ems.plot_matching_data(['Li$_2$S', 'Ni$_3$S$_2$'],'Ni3S2_Li2S_more.jpg', show_millers = True, show_legend = True)

# Define interface structural parameters 

In [ ]:
#These are defualt settings
iw.parse_interface_structure_params(termination_ftol = 0.01, c_periodic = False, \
                                    vacuum_over_film = 8, film_thickness = 10, \
                                    substrate_thickness = 10, shift_to_bottom = True)
#This will also screen out all the identical terminations

# Baysian Optimization by MLIP for Rigid Body Translation (RBT)

In [ ]:
#define structure optimization conditions (optional). If not called, the non-optimized energy and structure will be used
iw.parse_optimization_params(do = True, fix_shell = True, fix_in_layers = True, fix_thickness = 1)

do: whether to apply structure optimization for the sampled structure with minimum non-optimized energy  
fix_shell: whether to fix the bottom of the substrate  
fix_in_layers: whether to specify number of layers to set fix thickness  
fix_thickness: thickness to fix (in angstrom or in layers)  

In [ ]:
#Atomate & Fireworks workflow parameters (optional)
project_name = 'Li2S_Ni3S2'
db_file = '/public5/home/t6s001944/.conda/envs/general/lib/python3.12/site-packages/atomate/config/db.json'
vasp_cmd = 'mpirun -np 96 vasp_std'
user_incar_settings = {'NCORE':12}
work_dir = '/public5/home/t6s001944/InterOptimusPaper/mlip_benchmark/Li2S_Ni3S2/vasp'

In [ ]:
#MLIP docker container settings (optional)
user_settings = {'ncore':12, 'mem':18, 'timeout':3600}

Optimization settings:  
do: whether do structure relaxation by MLIP  
fix_shell: 0 no fixing, 1 only fixing the substrate surface, 2 fixing both substrate and film surfaces (recommended)  
fix_mode: 1 fixing xy-coordinates, 2 fixing xyz-coordinates (recommend)

In [ ]:
iw.parse_optimization_params(do = True, fix_shell = 2, fix_mode = 2)

n_calls: number of Bayesian sampling  
discut: structures with atoms having distances closer than this value will be calculated as having 0 energy (exclude this structure)  
dp: whether to do dipole correction for the Atomate & Fireworks VASP workflow

In [ ]:
#for matches
    #for terminations
        #for RBTs
iw.mlip_benchmark(['orb-models'], 
                  n_calls = 20, 
                  dp = False, 
                  discut = 0.8, 
                  user_settings = user_settings)

In [14]:
iw.global_optimized_data

,$h_s$,$k_s$,$l_s$,$h_f$,$k_f$,$l_f$,$A$ (Å$^2$),$\epsilon$,$E_{it}$ $(J/m^2)$,$E_{bd}$ $(J/m^2)$,...,$w_{f2}$,$u_{s1}$,$v_{s1}$,$w_{s1}$,$u_{s2}$,$v_{s2}$,$w_{s2}$,$T$,$i_m$,$i_t$
1,1,1,0,1,0,0,16.076353,0.006540,0.032478,-4.122752,...,1/2,-1/3,1/3,1/3,2/3,1/3,1/3,"(1_S_P4/mmm_1, 2_Ni_P4/mmm_1)",0,1
18,1,0,-1,1,0,1,45.470794,0.003387,0.559170,-1.379857,...,1/2,-2/3,2/3,-1/3,2/3,4/3,1/3,"(1_Li2S_Pmmm_3, 1_Ni_Pmmm_1)",3,0
10,1,0,-1,1,0,1,22.735397,0.003387,0.605502,-1.330055,...,0,2/3,1/3,1/3,0,1,0,"(1_Li2S_Pmmm_3, 1_Ni_Pmmm_1)",1,0
3,1,1,0,1,0,0,16.076353,0.006540,0.625695,-3.095544,...,1/2,-1/3,1/3,1/3,2/3,1/3,1/3,"(1_S_P4/mmm_1, 4_Ni_P4/mmm_1)",0,3
24,1,0,-1,1,0,-1,45.470794,0.009700,0.735145,-1.235571,...,1/2,-4/3,-2/3,1/3,0,0,1,"(1_Li2S_Pmmm_3, 2_Ni_Pmmm_1)",4,1
16,1,0,-1,1,0,-1,22.735397,0.009700,0.756492,-1.219246,...,0,2/3,1/3,1/3,-2/3,-1/3,2/3,"(1_Li2S_Pmmm_3, 2_Ni_Pmmm_1)",2,1
0,1,1,0,1,0,0,16.076353,0.006540,0.866916,-2.078045,...,1/2,-1/3,1/3,1/3,2/3,1/3,1/3,"(1_S_P4/mmm_1, 1_Ni_P4/mmm_1)",0,0
20,1,0,-1,1,0,1,45.470794,0.003387,0.929026,-0.827112,...,1/2,-2/3,2/3,-1/3,2/3,4/3,1/3,"(1_Li2S_Pmmm_3, 3_S_Pmmm_1)",3,2
12,1,0,-1,1,0,1,22.735397,0.003387,0.950408,-0.804805,...,0,2/3,1/3,1/3,0,1,0,"(1_Li2S_Pmmm_3, 3_S_Pmmm_1)",1,2
7,1,1,0,1,0,0,16.076353,0.006540,1.087337,-2.632887,...,1/2,-1/3,1/3,1/3,2/3,1/3,1/3,"(2_Li_P4/mmm_2, 3_S_P4/mmm_1)",0,7


Definition of the interface energy $E_{it}$ and binding energy $E_{bd}$ see:  
doi:10.26434/chemrxiv-2024-hwthh

# Get the interface with lowest predicted energy by MLIP for each match

In [15]:
iw.global_optimized_data.to_csv('all_data.csv')

In [16]:
import shutil
import os
try:
    shutil.rmtree('all_its')
except:
    pass
os.mkdir('all_its')

In [17]:
ids = iw.global_optimized_data.index.to_numpy()
i_s = iw.global_optimized_data['$i_m$'].to_numpy()
j_s = iw.global_optimized_data['$i_t$'].to_numpy()

match_ids = []
pairs = []
for i in range(len(i_s)):
    if i_s[i] not in match_ids:
        match_ids.append(i_s[i])
        pairs.append((i_s[i], j_s[i]))

for i in range(len(ids)):
    iw.benchmk_dict['orb-models'][(i_s[i],j_s[i])]['best_it']['structure'].to_file(f'all_its/{ids[i]}_POSCAR')

In [18]:
lowest_it_each_match = {}

In [30]:
try:
    shutil.rmtree('lowest_it_each_match')
except:
    pass
os.mkdir('lowest_it_each_match')

In [34]:
types = []
it_Es = []
bd_Es = []
for i in pairs:
    lowest_it_each_match[i[0]] = {}
    lowest_it_each_match[i[0]]['A'] = iw.benchmk_dict['orb-models'][i]['A']
    lowest_it_each_match[i[0]]['it_E'] = iw.benchmk_dict['orb-models'][i]['best_it']['it_E']
    lowest_it_each_match[i[0]]['bd_E'] = iw.benchmk_dict['orb-models'][i]['best_it']['bd_E']
    types.append(i[0])
    it_Es.append(lowest_it_each_match[i[0]]['it_E'])
    bd_Es.append(lowest_it_each_match[i[0]]['bd_E'])
    iw.benchmk_dict['orb-models'][i]['best_it']['structure'].to_file(f'lowest_it_each_match/{i[0]}_it_POSCAR')
    for j in ['stsg', 'stdb', 'fmsg', 'fmdb']:
        iw.benchmk_dict['orb-models'][i]['slabs'][j]['structure'].to_file(f'lowest_it_each_match/{i[0]}_{j}_POSCAR')

In [28]:
import numpy as np

In [29]:
np.savetxt('it_bd_Es.dat',np.column_stack((types, it_Es, bd_Es)), fmt = '%i %.4f %.4f')